In [1]:
from finn.util.visualization import showInNetron

from finn.util.test import get_test_model_trained
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

import numpy as np
import qonnx.core.onnx_exec as oxe

import os
import pandas as pd
import cv2

import my_metrics as metrics

# Load Cleaned and FINN Model

In [2]:
#ori_onnx = './BED_classifier__best_mean_F1.onnx'
# clean_onnx = './BED_classifier__best_mean_F1_clean.onnx'
# finn_onnx = './BED_classifier__best_mean_F1_finn.onnx'
clean_onnx = './BED_classifier__best_mean_F1_clean__QONNX.onnx'
finn_onnx = './BED_classifier__best_mean_F1_finn__QONNX.onnx'

In [3]:
#ori_model = ModelWrapper(ori_onnx)
clean_model = ModelWrapper(clean_onnx)
finn_model = ModelWrapper(finn_onnx)

## Execute with Dummy IN

In [4]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)

In [5]:
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(clean_model, input_dict)
produced_clean_qonnx = output_dict[list(output_dict.keys())[0]]
produced_clean_qonnx

array([[-1.8795719 , -0.48660603]], dtype=float32)

In [6]:
print(output_dict['global_out'][0])
print(output_dict[list(output_dict.keys())[0]][0])

[-1.8795719  -0.48660603]
[-1.8795719  -0.48660603]


# Eval with DFire MINI

In [17]:
DS_FOLDER = '../../datasets/dfire_mini/'
DS_FOLDER += 'test/'
DS_IMGS = DS_FOLDER + 'images/'
DS_LABELS = DS_FOLDER + 'labels/'

In [18]:
IMGS_LIST = [DS_IMGS + img_file for img_file in os.listdir(DS_IMGS)]

In [19]:
len(IMGS_LIST)

30

### Cleaned Model

In [20]:
clean_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = clean_model)

Number of test samples: 30



Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:03<00:00,  8.47it/s]


In [21]:
clean_df = pd.DataFrame(clean_metrics)
clean_df

,Smoke,Fire,Mean
Accuracy,0.7667,0.9667,0.8667
Precision,0.9412,0.9167,0.9290
Recall,0.7273,1.0000,0.8636
F1,0.8205,0.9565,0.8885


### FINN Model

In [22]:
finn_metrics = metrics.eval_quant_onnx(
    imgs_list = IMGS_LIST,
    labels_dir = DS_LABELS,
    model_wrapped = finn_model)

Number of test samples: 30



Testing:   0%|                                                                                                                                       | 0/30 [00:00<?, ?it/s]/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/util/basic.py:296: UserWarning: The values of tensor MultiThreshold_0_out0 can't be represented with the set datatype annotation (INT8), they will be rounded to match the datatype annotation.
  warnings.warn(
Testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [05:39<00:00, 11.31s/it]


In [23]:
finn_df = pd.DataFrame(finn_metrics)
finn_df

,Smoke,Fire,Mean
Accuracy,0.7667,0.9667,0.8667
Precision,0.9412,0.9167,0.9290
Recall,0.7273,1.0000,0.8636
F1,0.8205,0.9565,0.8885
